# Εργασία Μαθήματος Επιχειρησιακής Έρευνας<br>
## Ιωάννης Ορθοδόξου ΑΕΜ: 10822 <br>
### Σχεδιασμός Εβδομαδιαίου Προγράμματος Μαθημάτων

Το πρώτο πρόβλημα που έχουμε να λύσουμε, είναι ένα πρόβλημα δυαδικού προγραμματισμού αφού μπορούμε να χρησιμοποιήσουμε μεταβλητές απόφασης που είναι δύαδικές. Θα ορίσουμε τις μεταβλητές απόφασης ως εξής: 
$$X_{Teacher,Class,Day,Hour}$$
όπου για κάθε καθηγητή, για κάθε τμήμα, για κάθε ώρα και μέρα θα έχουμε μια μεταβλητή η όποια θα είναι 1 όταν ο καθηγητής διδάσκει στο συγκεκριμένο τμήμα, την συγκεκριμένη ώρα και μέρα. Σε αντίθετη περίπτωση η μεταβλητή μας θα παίρνει την τιμή 0.

In [17]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
model1 = gp.Model("Weekly_Timetable")
model1.setParam('OutputFlag', 0) # Suppress all Gurobi logs

Παρακάτω προσθέτουμε τα δέδομένα μας έτσι όπως δίνονται στην εκφώνηση της εργασίας. Η μεταβλητή teacher_hours περιέχει τα ονόματα των καθηγητών και τις ώρες που διδάσκουν σε κάθε τμήμα. Η λίστα classes περιέχει τα δύο τμήματα, οι πέντε μέρες της εβδομάδας περιέχονται στην λίστα days, ενώ στην λίστα hours περιέχονται οι χρονικές ζώνες όπου διδάσκονται τα μαθήματα. Για παράραδειγμα η πρώτη χρονική ζώνη αντιστοιχεί στις ώρες 8:00 - 10:00.

In [18]:
teachers_hours = {'Gesmanidis': [1, 1],
                 'Insoulina': [3, 3],
                 'Xartoula': [2, 2],
                 'Lathopraksis': [0, 4], 
                 'Antiparagogos': [4, 0], 
                 'Kirkofidou': [3, 3], 
                 'Platiazon': [1, 1], 
                 'Mpratsakis': [1, 0], 
                 'Trexalitoula': [0, 1]
}

lessons = ['English',
           'Biology',
           'Hist - Geo',
           'Mathematics',
           'Mathematics',
           'Physics',
           'Philosophy',
           'PE',
           'PE']

classes = ['Class_1', 'Class_2']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
hours = [1,2,3,4]

Έπειτα, θα εισάγουμε στο μοντέλο μας όλες τις μεταβλητές απόφασης.

In [19]:
x = {}
for teacher, (hours_per_class) in teachers_hours.items():
    for class_ in classes:
        for day in days:
            for hour in hours:
                x[teacher, class_, day, hour] = model1.addVar(
                    vtype=GRB.BINARY,
                    name=f"x[{teacher},{class_},{day},{hour}]"
                )

Ο πρώτος περιορισμός που έχουμε είναι ότι ο κάθε καθηγητής πρέπει να καλύψει τις ώρες διδασκαλίας που του αναλόγουν στο κάθε τμήμα και να μην διδάσκει ταυτόχρονα την ίδια μερα και ώρα και στα 2 τμήματα. Ακόμη δεν πρέπει δύο οι περισσότεροι καθηγητές να διδάσκουν την ίδια ώρα και μέρα στο ίδιο τμήμα. Επομένως, ο πρώτος περιορισμός θα είναι το άθροισμα των μετάβλητων που έχουν ίδιο καθηγητή και ίδιο τμήμα να είναι ίσες με τις ώρες που αναλογούν στο συγκεκριμένο τμήμα από τον συγκεκριμένο καθηγητή. Ο επόμενος περιορισμός, είναι το άθροισμα των μεταβλήτων που έχουν ίδιο καθηγητή, ίδια ώρα και μέρα αλλά διαφορετικό τμήμα το οποίο θα είναι μικρότερο ή ίσο με 1. Ο τρίτος περιορισμός θα είναι το άθροισμα των μεταβλητών που έχουν διαφορετικό καθηγητή, ίδια ώρα και μέρα και ίδιο τμήμα το οποίο θα είναι μικρότερο ή ίσο με 1.
$$\sum_{d}^{5}\sum_{h}^{4}X_{T,C,d,h} = required\_hours_{T} \quad \forall \space T,C$$ 
$$\sum_{c}^{2}X_{T,c,D,H} <= 1 \quad \forall T,D,H$$
$$\sum_{t}^{9}X_{t,C,D,H} <= 1 \quad \forall \space C,D,H$$
με T = Teacher, C = Class, D = Day, H = Hour

In [20]:
# 1 Constraint:
for teacher, (hours_per_class) in teachers_hours.items():
    for i, class_ in enumerate(classes):
        required_hours = hours_per_class[i]
        model1.addConstr(
                gp.quicksum(
                    x[teacher, class_, d, h]
                    for d in days for h in hours
                ) == required_hours,
                name=f"hours_{teacher}_{class_}"
            )

# 2 Constraint:
for teacher in teachers_hours.keys():
    for day in days:
        for hour in hours:
            model1.addConstr(
                gp.quicksum(x[teacher, class_, day, hour]
                            for class_ in classes
                ) <= 1,
                name=f"no_overlap_{teacher}_{day}_{hour}"
            )

# 3 Constraint:
for class_ in classes:
    for day in days:
        for hour in hours:
            model1.addConstr(
                gp.quicksum(x[teacher, class_, day, hour]
                            for teacher in teachers_hours.keys()
                )<= 1,
                name=f"one_teacher_per_hour_{class_}_{day}_{hour}"
            )

Ο επόμενος περιορισμός είναι το μάθημα τις φυσικής αγωγής να διδάσκεται την Πέμπτη μόνο την 3η ζώνη ώρας (14:00 - 16:00). Ο περιορισμός αυτός είναι η μεταβλητη απόφασης που έχει τον κ.Μπρατσάκη, το τμήμα 1 την Πέμπτη στην τρίτη ζώνη ώρας να ισούται με 1 και αντίστοιχα την κ.Τρεχαλητούλα, το τμήμα 2 την Πέμπτη στην τρίτη ζώνη ώρας να ισούται με 1.
$$X_{Mpratsakis,Class_1,Thursday,3} = 1 $$ 
$$X_{Trexalitoula,Class_2,Thursday,3} = 1 $$ 

In [21]:
# 4 Constraint:
model1.addConstr(x['Mpratsakis','Class_1','Thursday',3] == 1,
                 name=f"PE_restriction_Mpratsakis_Class_1_Thursday_3")

model1.addConstr(x['Trexalitoula','Class_2','Thursday',3] == 1,
                 name=f"PE_restriction_Trexalitoula_Class_2_Thursday_3")
model1.update()

Για την οργάνωση μελέτης της εβδομάδας η πρώτη ζώνη ώρας το πρώι της Δευτέρας είναι κρατημένη και επομένως <b>δεν θα βάλουμε κάποιο μάθημα</b>. Άρα ο περιορισμός που προκύπτει είναι η μεταβλητή που περιέχει την Δευτέρα και την πρώτη ζώνη ώρας, για όλους του καθηγητές και στα δύο τμήματα να είναι 0. 
$$X_{T,C,Monday,1} = 0 \quad \forall \space T,C$$ 

In [22]:
# 5 Constraint:
for teacher in teachers_hours.keys():
    for class_ in classes:
         model1.addConstr(x[teacher, class_, 'Monday', 1] == 0,
                          name=f"no_mon1_{teacher}_{class_}")

Ο επόμενος περιορισμός είναι είναι πως ο κ.Λαθοπράξης απουσιάζει κάθε Δευτέρα πρωί και επομένως δεν θα μπορέι να διδάσκει. Άρα η μεταβλητή που έχει τον κ.Λαθοπράξη στο τμήμα 2 την Δευτέρα με τις 2 πρώτες ζώνες ώρας θα πρέπει να είναι 0. 
$$X_{Lathopraksis,Class_2,Monday,H} = 0 \quad  \space ,H \in [1,2]$$ 

In [23]:
# 6 Constraint:
for hour in [1, 2]:
    model1.addConstr(x['Lathopraksis', 'Class_2', 'Monday', hour] == 0,
                     name=f"no_work_Lathospraksis_Mon_{class_}_{hour}")

Ο περιορισμός για το ότι η κ.Ινσουλίνα δεν εργάζεται τις Τετάρτες είναι ότι η μεταβλητή που έχει την κ.Ινσουλίνα και την Τετάρτη για οποιαδήποτε ώρα και τμήμα θα πρέπει να είναι 0. 
$$X_{Insoulina,C,Wednesday,H} = 0 \quad \forall \space C,H$$ 

In [24]:
# 7 Constraint:
for class_ in classes:
    for hour in hours:
        model1.addConstr(x['Insoulina', class_, 'Wednesday', hour] == 0,
                         name=f"no_work_Insoulina_Wed_{class_}_{hour}")

Τέλος για την αποφυγεί της πλήξης των μαθήτων δεν πρέπει να έχουν περισσότερες από 1 φορά με τον ίδιο καθηγητή στην ίδια μέρα. Επομένως θέλουμε το άθροισμα των μετάβλητών με τον ίδιο καθηγητή την ίδιο τμήμα, ίδια μέρα αλλά διαφορετική ώρα να είναι μικρότερο ή ίσο με 1.
$$\sum_{h}^{4}X_{T,C,D,h} <= 1 \quad \forall \space T,C,D $$ 

In [25]:
# 8 Constraint:
for teacher in teachers_hours.keys():
    for class_ in classes:
        for day in days:
            model1.addConstr(
                gp.quicksum(
                    x[teacher, class_, day, hour]
                    for hour in hours
                    if (teacher, class_, day, hour) in x
                ) <= 1,
                name=f"one_lesson_per_day_{teacher}_{class_}_{day}"
            )

Έπειτα αφού δεν έχουμε κάποιο συγκεκριμένο στόχο για να ελαχιστοποιήσουμε ή να μεγιστοποιήσουμε δεν έχουμε και αντικειμενική συνάρτηση και τρέχουμε απλά την συνάρτηση optimaze() έτσι ώστε να βρούμε μια εφικτή λύση.

In [26]:
model1.optimize()
    
programs = {class_: pd.DataFrame('', index=hours, columns=days) for class_ in classes}

# --- Εκτύπωση Λύσης ---
if model1.status == GRB.OPTIMAL:
    for key, var in x.items():
        if var.X > 0.5:
            teacher, class_, day, hour = key
            cell_value = f"{lessons[list(teachers_hours.keys()).index(teacher)]}"
            programs[class_].loc[hour, day] = cell_value
else:
    print("Δεν βρέθηκε λύση")

for class_, df in programs.items():
    print(f"\nTimetable {class_}:\n")
    print(df.to_string())


Timetable Class_1:

        Monday     Tuesday    Wednesday     Thursday       Friday
1               Hist - Geo      Physics      Biology  Mathematics
2      Physics     Physics   Philosophy   Hist - Geo             
3  Mathematics              Mathematics           PE      English
4      Biology                           Mathematics      Biology

Timetable Class_2:

        Monday      Tuesday    Wednesday     Thursday      Friday
1                   Biology  Mathematics                  Biology
2   Hist - Geo  Mathematics               Mathematics            
3      Biology   Philosophy                        PE     Physics
4  Mathematics      Physics      Physics      English  Hist - Geo


Όπως μπορούμε να δούμε από το αποτέλεσμα που έχουμε ικανοποιούνται όλοι οι χρονικοί περιορισμοί των μαθημάτων αλλά και οι απουσίες των καθηγητών.  

### Βέλτιστη Χωροθέτηση Αποθηκών <br>
Το δεύτερο πρόβλημα που έχουμε να λύσουμε είναι ένα προβλημα ακέραιου προγραματισμού.
Αρχικά θα εισάγουμε τα δεδομένα που μας δίνει η εκφώνηση. Σε ένα πίνακα θα χρειαστεί να υπολογίσουμε το κόστος μεταφοράς ανά τόνο απο μία αποθήκη σε ένα κέντρο πώλησης. Για να το κάνουμε αυτό, διαιρούμε το κόστος για την κάλυψη της συνολικής ζήτησης κάθε κέντρου πώλησης από μία αποθήκη με τη ζήτηση της αποθήκης.

In [27]:
warehouses = range(12)
centers = range(12)

costs_install = [3500, 9000, 10000, 4000, 3000, 9000, 9000, 3000, 4000, 10000, 9000, 3500]

capacity = [300, 250, 100, 180, 275, 300, 200, 220, 270, 250, 230, 180]

demand = [120, 80, 75, 100, 110, 100, 90, 60, 30, 150, 95, 120] 

INF = 1e9

cost_trans_all = [
    [100, 80 , 50 , 50 , 60 , 100, 120, 90 , 60 , 70 , 65 , 110],
    [120, 90 , 60 , 70 , 65 , 110, 140, 110, 80 , 80 , 75 , 130],
    [140, 110, 80 , 80 , 75 , 130, 160, 125, 100, 100, 80 , 150],
    [160, 125, 100, 100, 80 , 150, 190, 150, 130, INF, INF, INF],
    [190, 150, 130, INF, INF, INF, 180, 150, 50 , 50 , 60 , 100],
    [200, 180, 150, INF, INF, INF, 100, 120, 90 , 60 , 75 , 110],
    [120, 90 , 60 , 70 , 65 , 110, 140, 110, 80 , 80 , 75 , 130],
    [120, 90 , 60 , 70 , 65 , 110, 140, 110, 80 , 80 , 75 , 130],
    [140, 110, 80 , 80 , 75 , 130, 160, 125, 100, 100, 80 , 150],
    [160, 125, 100, 100, 80 , 150, 190, 150, 130, INF, INF, INF],
    [190, 150, 130, INF, INF, INF, 200, 180, 150, INF, INF, INF],
    [200, 180, 150, INF, INF, INF, 100, 80 , 50 , 50 , 60 , 100]
]

cost_trans_per_ton = [[0 for j in range(12)] for i in range(12)]
for i in warehouses:
    for j in centers:
        cost_trans_per_ton[i][j] = cost_trans_all[i][j] / demand[j]

Έπειτα θα εισάγουμε τις μεταβλητές απόφασεις στο μοντέλο μας. Οι μεταβλητές απόφασεις θα είναι: 
* $ y_i $ = Η αποθήκη i θα ανοίξει ή όχι και θα είναι δυαδική μεταβλητη  
* $ x_{ij} $ = Τόνοι που στέλνει η αποθήκη i στο κέντρο j και επομένως θα είναι συνεχής μεταβλητη. 

In [28]:
model2 = gp.Model("Warehouse_Location")
model2.setParam('OutputFlag', 0) # Suppress all Gurobi logs
y = model2.addVars(warehouses, vtype=GRB.BINARY, name="y")

x = model2.addVars(warehouses,centers, vtype=GRB.CONTINUOUS, name="x")

Ένας περιορισμός που έχουμε είναι πως η ζήτηση κάθε κέντρου πώλησης θα καλύπτεται πλήρως. Αυτός ο περιορισμός είναι το άθροισμα όλων των μεταβλητων x με ίδιο j (κέντρο) για όλα τα i (αποθήκες), να είναι ίσο με την ζήτηση του κέντρου.
$$\sum_{i=1}^{12}x_{ij} = demand_j \quad \forall j $$

In [29]:
for center in centers:
    model2.addConstr(
        gp.quicksum(
            x[warehouse,center] 
            for warehouse in warehouses
        ) == demand[center],
        name=f"demand_{center}"
    )

Ένας άλλος περιορισμός που έχουμε είναι ότι η ποσότητα που στέλνει η αποθήκη δεν μπορεί να ξεπερνάει την χωρητικότητα της. Αυτός ο περιορισμός είναι το άθροισμα όλων των μεταβλητών x με ίδιο i (αποθήκη) για όλα τα j (κέντρα), να είναι ίσο ή μικρότερο της χωρήτικότητας της αποθήκης πολλαπλασιασμένη με την μεταβλητή $y_i$ 
$$\sum_{j=1}^{12}x_{ij} \leq capacity_i \cdot y_i\quad \forall i $$

In [30]:
for warehouse in warehouses:
    model2.addConstr(
        gp.quicksum(
            x[warehouse,center] 
            for center in centers
        ) <= capacity[warehouse] * y[warehouse], name=f'capacity_{i}'
    )

Καταλήγουμε στο ότι θέλουμε να ελαχιστοποιήσουμε το κόστος και άρα να ελαχιστοποιήσουμε την αντικειμενική συναρτηση η οποία είναι τα αθροίσματα των κοστών: 
$$Minimize \sum_{i}cost\_install_i \cdot y_i + \sum_{i,j} cost\_per\_ton_{i,j} \cdot x_{ij}$$ 

In [31]:
model2.setObjective(
    gp.quicksum(
        costs_install[i] * y[i]
        for i in warehouses) +
    gp.quicksum(
        cost_trans_per_ton[i][j] * x[i,j]
        for i in warehouses for j in centers
    ),
    GRB.MINIMIZE 
)

model2.optimize()

if model2.status == GRB.OPTIMAL:
    print("Βρέθηκε βέλτιστη λύση.")
elif model2.status == GRB.INFEASIBLE:
    print("Το μοντέλο είναι αδύνατο.")


for i in warehouses:
    if y[i].X > 0.5:
        print(f"Αποθήκη {i+1} άνοιξε")

for i in warehouses:
    for j in centers:
        if x[i,j].X > 0:
            print(f"Αποθήκη {i+1} στέλνει {x[i,j].X:.2f} τόνους στο κέντρο {j+1}")

print(f"Συνολικό κόστος: {model2.ObjVal:.2f}")


Βρέθηκε βέλτιστη λύση.
Αποθήκη 1 άνοιξε
Αποθήκη 5 άνοιξε
Αποθήκη 8 άνοιξε
Αποθήκη 9 άνοιξε
Αποθήκη 12 άνοιξε
Αποθήκη 1 στέλνει 120.00 τόνους στο κέντρο 1
Αποθήκη 1 στέλνει 5.00 τόνους στο κέντρο 2
Αποθήκη 1 στέλνει 75.00 τόνους στο κέντρο 3
Αποθήκη 1 στέλνει 100.00 τόνους στο κέντρο 4
Αποθήκη 5 στέλνει 150.00 τόνους στο κέντρο 10
Αποθήκη 5 στέλνει 5.00 τόνους στο κέντρο 11
Αποθήκη 5 στέλνει 120.00 τόνους στο κέντρο 12
Αποθήκη 8 στέλνει 75.00 τόνους στο κέντρο 2
Αποθήκη 8 στέλνει 45.00 τόνους στο κέντρο 5
Αποθήκη 8 στέλνει 100.00 τόνους στο κέντρο 6
Αποθήκη 9 στέλνει 65.00 τόνους στο κέντρο 5
Αποθήκη 9 στέλνει 90.00 τόνους στο κέντρο 11
Αποθήκη 12 στέλνει 90.00 τόνους στο κέντρο 7
Αποθήκη 12 στέλνει 60.00 τόνους στο κέντρο 8
Αποθήκη 12 στέλνει 30.00 τόνους στο κέντρο 9
Συνολικό κόστος: 17929.23


In [32]:
for i in warehouses:
    total_sent = sum(x[i,j].X for j in centers)
    cap = capacity[i]
    print(f"Αποθήκη {i+1}: Αποσταλμένη ποσότητα = {total_sent:.2f}, Χωρητικότητα = {cap}")
    if total_sent > cap:
        print(f"ΠΡΟΒΛΗΜΑ: Υπέρβαση χωρητικότητας!")

for j in centers:
    total_received = sum(x[i,j].X for i in warehouses)
    demand_j = demand[j]
    print(f"Κέντρο {j+1}: Ληφθείσα ποσότητα = {total_received:.2f}, Ζήτηση = {demand_j}")
    if abs(total_received - demand_j) > 0:
        print(f"ΠΡΟΒΛΗΜΑ: Ζήτηση μη καλυφθείσα ή υπερκαλυφθείσα!")

for i in warehouses:
    if y[i].X < 0.5:
        total_sent = sum(x[i,j].X for j in centers)
        if total_sent > 0:
            print(f"ΠΡΟΒΛΗΜΑ: Αποθήκη {i+1} είναι κλειστή αλλά στέλνει {total_sent:.2f}")


Αποθήκη 1: Αποσταλμένη ποσότητα = 300.00, Χωρητικότητα = 300
Αποθήκη 2: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 250
Αποθήκη 3: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 100
Αποθήκη 4: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 180
Αποθήκη 5: Αποσταλμένη ποσότητα = 275.00, Χωρητικότητα = 275
Αποθήκη 6: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 300
Αποθήκη 7: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 200
Αποθήκη 8: Αποσταλμένη ποσότητα = 220.00, Χωρητικότητα = 220
Αποθήκη 9: Αποσταλμένη ποσότητα = 155.00, Χωρητικότητα = 270
Αποθήκη 10: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 250
Αποθήκη 11: Αποσταλμένη ποσότητα = 0.00, Χωρητικότητα = 230
Αποθήκη 12: Αποσταλμένη ποσότητα = 180.00, Χωρητικότητα = 180
Κέντρο 1: Ληφθείσα ποσότητα = 120.00, Ζήτηση = 120
Κέντρο 2: Ληφθείσα ποσότητα = 80.00, Ζήτηση = 80
Κέντρο 3: Ληφθείσα ποσότητα = 75.00, Ζήτηση = 75
Κέντρο 4: Ληφθείσα ποσότητα = 100.00, Ζήτηση = 100
Κέντρο 5: Ληφθείσα ποσότητα = 110.00, Ζήτηση = 110
Κέντρο 6: Ληφθείσα ποσότητα 

Όπως μπορουμε να δούμε όλες οι αποθήκες στέλνουν μικρότερη ή ιση ποσότητα με αυτή της χωρητικότητας τους και όλα τα κέντρα πώλησης λαμβάνουν την ποσότητα την όποια ζητούν. 